## Evaluierung ChromaDB

In diesem Notebook werden die verschiedenene Parameter von ChromaDB gelistet. Dabei werden verschiedene Datenbänke erstellt und miteinenader verglichen.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from Chroma_Embedding_Pipeline import ChromaEmbeddingPipeline
from chromadb.api.client import SharedSystemClient
from langchain.embeddings import AzureOpenAIEmbeddings
import chromadb
import pandas as pd
import os
import credentials
from itertools import product

In [2]:
os.environ["AZURE_OPENAI_API_KEY"] = credentials.AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = credentials.AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"

In [3]:
# Bereinigte Daten einlesen
df_train_cleaned = pd.read_parquet('../data_mc1/data_processed/df_train_1000.parquet')

In [4]:
df_train_cleaned.head()

,id,title,date,domain,url,language,content
0,93950,Agrivoltaics Goes Nuclear On California Prairie,2022-12-18,cleantechnica,https://cleantechnica.com/2022/12/18/agrivolta...,en,Agrivoltaics Goes Nuclear On California Prairi...
1,93986,The Wait For Hydrogen Fuel Cell Electric Aircr...,2023-01-02,cleantechnica,https://cleantechnica.com/2023/01/02/the-wait-...,en,The Wait For Hydrogen Fuel Cell Electric Aircr...
2,21630,Quality Control System for Ocean Temperature I...,2023-01-30,azocleantech,https://www.azocleantech.com/news.aspx?newsID=...,en,Quality Control System for Ocean Temperature I...
3,98795,European Commission introduces Green Deal Indu...,2023-02-02,pv-magazine,https://www.pv-magazine.com/2023/02/02/europea...,en,European Commission introduces Green Deal Indu...
4,21739,More Mild Winters with Erratic Temperature Cha...,2023-03-01,azocleantech,https://www.azocleantech.com/news.aspx?newsID=...,en,More Mild Winters with Erratic Temperature Cha...


In [7]:
embedding_models = [
    "text-embedding-ada-002",
    "text-embedding-3-large"
]

base_configs = [
    {
        "splitter_type": "RecursiveCharacterTextSplitter",
        "chunk_size": 800,
        "chunk_overlap": 100,
        "separators": ["\n\n", "\n", ".", " ", ""],
        "hnsw:space": "cosine",
        "collection_name": "recchar_800_cosine",
        "persist_directory": "./chroma_recchar_800_cosine"
    },
    {
        "splitter_type": "RecursiveCharacterTextSplitter",
        "chunk_size": 800,
        "chunk_overlap": 100,
        "separators": ["\n\n", "\n", ".", " ", ""],
        "hnsw:space": "l2",
        "collection_name": "recchar_800_l2",
        "persist_directory": "./chroma_recchar_800_l2"
    },
    {
        "splitter_type": "RecursiveCharacterTextSplitter",
        "chunk_size": 800,
        "chunk_overlap": 100,
        "separators": ["\n\n", "\n", ".", " ", ""],
        "hnsw:space": "ip",
        "collection_name": "recchar_800_ip",
        "persist_directory": "./chroma_recchar_800_ip"
    },
    {
        "splitter_type": "SentenceTransformersTokenTextSplitter",
        "chunk_size": 200,
        "chunk_overlap": 20,
        "model_name": "sentence-transformers/all-MiniLM-L6-v2",
        "hnsw:space": "cosine",
        "collection_name": "senttrans_200_cosine",
        "persist_directory": "./chroma_senttrans_200_cosine"
    },
    {
        "splitter_type": "SentenceTransformersTokenTextSplitter",
        "chunk_size": 200,
        "chunk_overlap": 20,
        "model_name": "sentence-transformers/all-MiniLM-L6-v2",
        "hnsw:space": "l2",
        "collection_name": "senttrans_200_l2",
        "persist_directory": "./chroma_senttrans_200_l2"
    },
    {
        "splitter_type": "SentenceTransformersTokenTextSplitter",
        "chunk_size": 200,
        "chunk_overlap": 20,
        "model_name": "sentence-transformers/all-MiniLM-L6-v2",
        "hnsw:space": "ip",
        "collection_name": "senttrans_200_ip",
        "persist_directory": "./chroma_senttrans_200_ip"
    }
]

# Gemeinsame Metadaten- und Pipeline-Parameter
metadata_config_base = {
    "hnsw:M": 16,
    "efConstruction": 200
}
pipeline_params = {
    "deployment_name": "text-embedding-3-large",
    "batch_size": 20,
    "sleep_time": 1.0
}

all_configs = []
for base in base_configs:
    for model in embedding_models:
        new_config = base.copy()
        new_config["deployment_name"] = model
        new_config["collection_name"] = f"{base['collection_name']}_{model.replace('text-embedding-', '')}"
        new_config["persist_directory"] = f"{base['persist_directory']}_{model.replace('text-embedding-', '')}"
        all_configs.append(new_config)

vector_stores = []

for config in all_configs:
    print(f"Processing config: {config['collection_name']}")
    
    SharedSystemClient.clear_system_cache()  # falls verwendet

    # TextSplitter erstellen
    if config["splitter_type"] == "RecursiveCharacterTextSplitter":
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=config["chunk_size"],
            chunk_overlap=config["chunk_overlap"],
            separators=config["separators"]
        )
    else:
        splitter = SentenceTransformersTokenTextSplitter(
            chunk_size=config["chunk_size"],
            chunk_overlap=config["chunk_overlap"],
            model_name=config["model_name"]
        )

    metadata_config = metadata_config_base.copy()
    metadata_config["hnsw:space"] = config["hnsw:space"]

    chroma_settings = chromadb.Settings(
        allow_reset=True,
        is_persistent=True,
        persist_directory=config["persist_directory"],
        anonymized_telemetry=False
    )

    pipeline = ChromaEmbeddingPipeline(
        df=df_train_cleaned,
        text_column="content",
        deployment_name=config["deployment_name"],
        splitter=splitter,
        collection_name=config["collection_name"],
        persist_directory=config["persist_directory"],
        metadata=metadata_config,
        batch_size=pipeline_params["batch_size"],
        sleep_time=pipeline_params["sleep_time"],
        chroma_settings=chroma_settings
    )

    vector_store = pipeline.run()
    vector_stores.append(vector_store)

    print(f"Created DB: {config['collection_name']}")


Processing config: recchar_800_cosine_ada-002
6394 Chunks erzeugt.


/Users/husazwerg/code/gitlab.fhnw.ch/gruppen_arbeit/npr_fs25/MC1/src/Chroma_Embedding_Pipeline.py:146: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


Batch 1: 20 Chunks hinzugefügt
Batch 2: 20 Chunks hinzugefügt
Batch 3: 20 Chunks hinzugefügt
Batch 4: 20 Chunks hinzugefügt
Batch 5: 20 Chunks hinzugefügt
Batch 6: 20 Chunks hinzugefügt
Batch 7: 20 Chunks hinzugefügt
Batch 8: 20 Chunks hinzugefügt
Batch 9: 20 Chunks hinzugefügt
Batch 10: 20 Chunks hinzugefügt
Batch 11: 20 Chunks hinzugefügt
Batch 12: 20 Chunks hinzugefügt
Batch 13: 20 Chunks hinzugefügt
Batch 14: 20 Chunks hinzugefügt
Batch 15: 20 Chunks hinzugefügt
Batch 16: 20 Chunks hinzugefügt
Batch 17: 20 Chunks hinzugefügt
Batch 18: 20 Chunks hinzugefügt
Batch 19: 20 Chunks hinzugefügt
Batch 20: 20 Chunks hinzugefügt
Batch 21: 20 Chunks hinzugefügt
Batch 22: 20 Chunks hinzugefügt
Batch 23: 20 Chunks hinzugefügt
Batch 24: 20 Chunks hinzugefügt
Batch 25: 20 Chunks hinzugefügt
Batch 26: 20 Chunks hinzugefügt
Batch 27: 20 Chunks hinzugefügt
Batch 28: 20 Chunks hinzugefügt
Batch 29: 20 Chunks hinzugefügt
Batch 30: 20 Chunks hinzugefügt
Batch 31: 20 Chunks hinzugefügt
Batch 32: 20 Chun

/Users/husazwerg/code/gitlab.fhnw.ch/gruppen_arbeit/npr_fs25/MC1/src/Chroma_Embedding_Pipeline.py:167: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


Gespeichert unter: ./chroma_recchar_800_cosine_ada-002
Created DB: recchar_800_cosine_ada-002
Processing config: recchar_800_cosine_3-large
6394 Chunks erzeugt.
Batch 1: 20 Chunks hinzugefügt
Batch 2: 20 Chunks hinzugefügt
Batch 3: 20 Chunks hinzugefügt
Batch 4: 20 Chunks hinzugefügt
Batch 5: 20 Chunks hinzugefügt
Batch 6: 20 Chunks hinzugefügt
Batch 7: 20 Chunks hinzugefügt
Batch 8: 20 Chunks hinzugefügt
Batch 9: 20 Chunks hinzugefügt
Batch 10: 20 Chunks hinzugefügt
Batch 11: 20 Chunks hinzugefügt
Batch 12: 20 Chunks hinzugefügt
Batch 13: 20 Chunks hinzugefügt
Batch 14: 20 Chunks hinzugefügt
Batch 15: 20 Chunks hinzugefügt
Batch 16: 20 Chunks hinzugefügt
Batch 17: 20 Chunks hinzugefügt
Batch 18: 20 Chunks hinzugefügt
Batch 19: 20 Chunks hinzugefügt
Batch 20: 20 Chunks hinzugefügt
Batch 21: 20 Chunks hinzugefügt
Batch 22: 20 Chunks hinzugefügt
Batch 23: 20 Chunks hinzugefügt
Batch 24: 20 Chunks hinzugefügt
Batch 25: 20 Chunks hinzugefügt
Batch 26: 20 Chunks hinzugefügt
Batch 27: 20 Chu

/Users/husazwerg/code/gitlab.fhnw.ch/gruppen_arbeit/npr_fs25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3807 Chunks erzeugt.
Batch 1: 20 Chunks hinzugefügt
Batch 2: 20 Chunks hinzugefügt
Batch 3: 20 Chunks hinzugefügt
Batch 4: 20 Chunks hinzugefügt
Batch 5: 20 Chunks hinzugefügt
Batch 6: 20 Chunks hinzugefügt
Batch 7: 20 Chunks hinzugefügt
Batch 8: 20 Chunks hinzugefügt
Batch 9: 20 Chunks hinzugefügt
Batch 10: 20 Chunks hinzugefügt
Batch 11: 20 Chunks hinzugefügt
Batch 12: 20 Chunks hinzugefügt
Batch 13: 20 Chunks hinzugefügt
Batch 14: 20 Chunks hinzugefügt
Batch 15: 20 Chunks hinzugefügt
Batch 16: 20 Chunks hinzugefügt
Batch 17: 20 Chunks hinzugefügt
Batch 18: 20 Chunks hinzugefügt
Batch 19: 20 Chunks hinzugefügt
Batch 20: 20 Chunks hinzugefügt
Batch 21: 20 Chunks hinzugefügt
Batch 22: 20 Chunks hinzugefügt
Batch 23: 20 Chunks hinzugefügt
Batch 24: 20 Chunks hinzugefügt
Batch 25: 20 Chunks hinzugefügt
Batch 26: 20 Chunks hinzugefügt
Batch 27: 20 Chunks hinzugefügt
Batch 28: 20 Chunks hinzugefügt
Batch 29: 20 Chunks hinzugefügt
Batch 30: 20 Chunks hinzugefügt
Batch 31: 20 Chunks hinzugef